<a href="https://colab.research.google.com/github/jcmachicao/MachineLearningAvanzado_UC_2024/blob/main/U3_Ejemplo_Arquitectura_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Definir la arquitectura del generador
class Generator(nn.Module):
    def __init__(self, latent_dim, image_shape):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.image_shape = image_shape

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh()  # Capa de salida con función de activación tanh para generar imágenes entre -1 y 1
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *self.image_shape)  # Remodelar la salida para que tenga la forma de una imagen
        return img

In [ ]:
# Definir la arquitectura del discriminador
class Discriminator(nn.Module):
    def __init__(self, image_shape):
        super(Discriminator, self).__init__()
        self.image_shape = image_shape

        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Capa de salida con función de activación sigmoide para clasificar entre real o falso
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)  # Aplanar la imagen para pasarla al discriminador
        validity = self.model(img_flat)
        return validity

In [ ]:
# Parámetros
latent_dim = 100
image_shape = (1, 28, 28)  # Tamaño de las imágenes MNIST

# Inicializar generador y discriminador
generator = Generator(latent_dim, image_shape)
discriminator = Discriminator(image_shape)

# Definir la función de pérdida y los optimizadores
adversarial_loss = nn.BCELoss()  # Pérdida adversarial utilizando la entropía cruzada binaria
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Configurar los datos de MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.MNIST(root='mnist_data/', train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
# Entrenamiento del GAN
num_epochs = 30
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Entrenar al discriminador
        optimizer_D.zero_grad()

        # Configurar las etiquetas para imágenes reales y falsas
        real_labels = torch.ones(imgs.size(0), 1)
        fake_labels = torch.zeros(imgs.size(0), 1)

        # Generar ruido aleatorio
        z = torch.randn(imgs.size(0), latent_dim)

        # Generar imágenes falsas
        gen_imgs = generator(z)

        # Evaluar imágenes reales y falsas con el discriminador
        real_loss = adversarial_loss(discriminator(imgs), real_labels)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake_labels)

        # Calcular la pérdida total del discriminador
        d_loss = (real_loss + fake_loss) / 2

        # Retropropagación y optimización
        d_loss.backward()
        optimizer_D.step()

        # Entrenar al generador
        optimizer_G.zero_grad()

        # Evaluar imágenes generadas por el generador con el discriminador
        validity = discriminator(gen_imgs)
        g_loss = adversarial_loss(validity, real_labels)

        # Retropropagación y optimización
        g_loss.backward()
        optimizer_G.step()

        # Imprimir progreso del entrenamiento
        print(
            f"Epoch [{epoch}/{num_epochs}], Batch [{i}/{len(dataloader)}], "
            f"Loss D: {d_loss.item():.4f}, Loss G: {g_loss.item():.4f}"
        )

        # Guardar imágenes generadas durante el entrenamiento
        if i % 400 == 0:
            torchvision.utils.save_image(gen_imgs.data[:25], f"gan_images/{epoch}_{i}.png", nrow=5, normalize=True)